# 使用argparse进行调参

## 1. argparse简介
将命令行传入的其他参数进行解析、保存和使用。在使用argparse后，我们在命令行输入的参数就可以以这种形式python file.py --lr 1e-4 --batch_size 32来完成对常见超参数的设置

## 2. argparse的使用
- 创建ArgumentParser()对象
- 调用add_argument()方法添加参数
- 使用parse_args()解析参数 在接下来的内容中，我们将以实际操作来学习argparse的使用方法。

In [ ]:
# demo.py
import argparse

# 创建ArgumentParser()对象
parser = argparse.ArgumentParser()

# 添加参数
parser.add_argument('-o', 
                    '--output', 
                    action='store_true', 
                    help="shows output")
# action = `store_true` 会将output参数记录为True
# type 规定了参数的格式
# default 规定了默认值
parser.add_argument('--lr', 
                    type=float, 
                    default=3e-5, 
                    help='select the learning rate, default=1e-3') 

parser.add_argument('--batch_size', 
                    type=int, 
                    required=True, 
                    help='input batch size')  

# 使用parse_args()解析函数
args = parser.parse_args()

if args.output:
    print("This is some output")
    print(f"learning rate:{args.lr} ")


```
python demo.py --lr 3e-4 --batch_size 32
```

```
# positional.py
import argparse

# 位置参数
parser = argparse.ArgumentParser()

parser.add_argument('name')
parser.add_argument('age')

args = parser.parse_args()

print(f'{args.name} is {args.age} years old')
```
当我们不实用--后，将会严格按照参数位置进行解析
```
positional_arg.py Peter 23
```

## 3. 高效使用argparse修改超参数

config.py文件保存超参数


In [ ]:
import argparse


def get_options(parser=argparse.ArgumentParser()):

    parser.add_argument('--workers',
                        type=int,
                        default=0,
                        help='number of data loading workers, you had better put it 4 times of your gpu')

    parser.add_argument('--batch_size',
                        type=int,
                        default=4,
                        help='input batch size, default=64')

    parser.add_argument('--niter',
                        type=int,
                        default=10,
                        help='number of epochs to train for, default=10')

    parser.add_argument('--lr',
                        type=float,
                        default=3e-5,
                        help='select the learning rate, default=1e-3')

    parser.add_argument('--seed',
                        type=int,
                        default=118,
                        help="random seed")

    parser.add_argument('--cuda',
                        action='store_true',
                        default=True,
                        help='enables cuda')

    parser.add_argument('--checkpoint_path',
                        type=str,
                        default='',
                        help='Path to load a previous trained model if not empty (default empty)')

    parser.add_argument('--output',
                        action='store_true',
                        default=True,
                        help="shows output")

    opt = parser.parse_args()

    if opt.output:
        print(f'num_workers: {opt.workers}')
        print(f'batch_size: {opt.batch_size}')
        print(f'epochs (niters) : {opt.niter}')
        print(f'learning rate : {opt.lr}')
        print(f'manual_seed: {opt.seed}')
        print(f'cuda enable: {opt.cuda}')
        print(f'checkpoint_path: {opt.checkpoint_path}')

    return opt


if __name__ == '__main__':
    opt = get_options()

```
$ python config.py

num_workers: 0
batch_size: 4
epochs (niters) : 10
learning rate : 3e-05
manual_seed: 118
cuda enable: True
checkpoint_path:
```

在train.py等其他文件，使用下面的结果调用参数

In [ ]:
# 导入必要库
import config
...

opt = config.get_options()

manual_seed = opt.seed
num_workers = opt.workers
batch_size = opt.batch_size
lr = opt.lr
niters = opt.niters
checkpoint_path = opt.checkpoint_path

# 随机数的设置，保证复现结果


def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


...


if __name__ == '__main__':
	set_seed(manual_seed)
	for epoch in range(niters):
		train(model, lr, batch_size, num_workers, checkpoint_path)
		val(model, lr, batch_size, num_workers, checkpoint_path)